## 数据加载
我们的误差补偿训练数据存放在<I>adjustments.tsv</I>中，测试数据放在<I>test_adjustments.tsv</I>。

**测试数据集为真实值，不能进行调整，否则会导致实际模型测试结果和实际的预测结果存在偏差，出现过于乐观或者过于消极的测试结果**

只需要执行加载训练数据的代码即可，当然也可以修改代码加载指定文件

In [1]:
import numpy as np
import pandas as pd

# 加载训练数据
train_dataset = pd.read_csv('./adjustments.tsv',
                          sep='\t',
                          skipinitialspace=True)
# 加载测试数据     ！！！ 测试数据集为真实值，不能进行调整，否则将会导致实际模型测试结果和真实预测结果存在偏差，使得最终加工的作品和预期不一致
test_dataset = pd.read_csv('./test_adjustments.tsv',
                          sep='\t',
                          skipinitialspace=True)

## 数据分析

打印并查看数据，可以看出，adjustments.tsv文件的前n列为特征值，这些值代表着真实世界中影响机床的环境因素，例如刀具磨损、温度、湿度等等；后面几列为补偿指令。

In [2]:
np.set_printoptions(precision=3, suppress=True)
dataset = train_dataset.copy()
dataset.head()


,特征0,特征1,特征2,特征3,特征4,特征5,特征6,特征7,特征8,特征9,...,特征16,特征17,补偿0,补偿1,补偿2,补偿3,补偿4,补偿5,补偿6,补偿7
0,1.4158,2.9711,10.7935,7.5279,2.3352,8.1042,2.3096,3.3367,11.8639,12.7142,...,171.764,1434.24,0.331511,-0.932553,0.285048,-0.1435,-0.833982,0.767568,0.463969,1.904800
1,0.6280,1.8616,10.1770,7.4684,2.1915,8.5945,0.1379,2.9661,11.5816,12.2487,...,185.824,1469.19,0.894066,-0.446796,0.058519,-0.4624,0.715252,0.999105,0.988844,0.689742
2,0.9648,1.8103,10.1682,5.9705,2.0629,6.5349,2.8694,3.1185,11.7464,12.2074,...,187.576,1540.76,0.999982,0.460716,0.997809,-0.4624,0.723031,0.992935,0.682903,0.749580
3,0.7119,1.6221,10.1487,6.8678,2.0694,6.8806,1.5791,2.3003,11.5545,12.0659,...,189.938,1498.29,0.998794,-0.862448,0.329694,-0.4624,0.891745,0.015078,0.997127,0.984439
4,0.3797,1.6852,10.9601,5.0035,3.1659,5.9471,0.0858,2.6402,11.7458,12.7041,...,181.275,1465.11,0.442893,-0.990703,-0.151400,0.2245,0.617880,-0.506397,-0.997502,0.376126


In [7]:
ds=dataset.dropna()
w=['特征'+str(i) for i in range(18)]
ds=ds.drop_duplicates(w)
print(ds.shape)
ds.describe()

ds1=test_dataset.dropna()
w=['特征'+str(i) for i in range(18)]
ds1=ds1.drop_duplicates(w)
print(ds1.shape)
ds1.describe()


(50000, 26)
(2998, 26)


,特征0,特征1,特征2,特征3,特征4,特征5,特征6,特征7,特征8,特征9,...,特征16,特征17,补偿0,补偿1,补偿2,补偿3,补偿4,补偿5,补偿6,补偿7
count,2998.000000,2998.000000,2998.000000,2998.000000,2998.000000,2998.000000,2998.000000,2998.000000,2998.000000,2998.000000,...,2998.000000,2998.000000,2998.000000,2998.000000,2998.000000,2998.000000,2998.000000,2998.000000,2998.000000,2998.000000
mean,1.008837,2.018090,10.501365,6.980399,2.991379,6.986831,1.983407,2.994988,11.739018,12.600981,...,172.242767,1474.643939,0.525279,0.005173,0.250365,0.010439,-0.269551,0.524640,0.222959,0.561442
std,0.573132,0.578950,0.286296,1.134771,0.574884,1.142659,1.144038,0.577503,0.128526,0.196063,...,12.198489,54.894522,0.399656,0.699439,0.497970,0.330564,0.672572,0.494762,0.716798,1.124138
min,0.000500,1.000000,10.000300,5.000700,2.000400,5.000800,0.000100,2.000700,11.155400,11.750600,...,121.207000,1377.980000,-0.950279,-0.999992,-1.240660,-0.533200,-1.000000,-0.863171,-0.999998,-3.354270
25%,0.518975,1.506825,10.257700,6.033325,2.501750,6.004525,1.001925,2.509325,11.668100,12.485025,...,165.299500,1437.355000,0.279705,-0.693314,-0.096690,-0.143500,-0.847946,0.307439,-0.492667,-0.107755
50%,1.014300,2.042550,10.497350,6.962800,2.955600,6.956750,1.985950,2.986350,11.762100,12.644450,...,174.141000,1456.825000,0.617192,0.015394,0.250205,-0.143500,-0.550868,0.709205,0.495988,0.390784
75%,1.488325,2.523625,10.750450,7.944000,3.481775,7.957600,2.953800,3.496425,11.835725,12.747475,...,181.637250,1496.542500,0.860046,0.696679,0.604809,0.224500,0.370577,0.905374,0.877369,1.159790
max,1.998200,2.998600,10.999300,8.996300,3.999400,8.998500,3.996200,3.999400,11.950100,12.933200,...,193.145000,1785.880000,1.000000,1.000000,1.832180,0.632400,0.999997,0.999997,1.000000,6.585870


In [8]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# # 选择需要可视化的数据列
# #data = ds[ds]

# # 循环遍历每个数据列
# for col in ds.columns:
#     # 绘制单个数据列的核密度估计图
#     fig = plt.figure(figsize=(5, 3))
#     sns.kdeplot(ds[col])
    
#     # 添加网格线
#     plt.grid()
#     #plt.xticks(np.arange(-0.2, 1.2, 0.1),rotation=45)
   
#     # 设置图的标题和横轴标签
#     plt.title(f'Kernel Density Estimate - {col}')
#     plt.xlabel(col)
    
#     # 显示绘制的图形
#     plt.show()

然后我们可以简单分析数据，比如我们可以查看特征值和补偿值的分布特性，比如均值和方差：

In [9]:
average = np.average(ds.values[:,:18], axis=0)
variance = np.var(ds.values[:,:18], axis=0)
print('均值', average)
print('方差', variance)

# average1 = np.average(ds1.values[:,:18], axis=0)
# variance1 = np.var(ds1.values[:,:18], axis=0)
# print('均值', average1)
# print('方差', variance1)


均值 [   1.       1.998   10.5      6.998    3.001    6.995    2.005    2.999
   11.736   12.604    1.768    0.649   19.417   19.598    3.349   17.983
  172.057 1475.054]
方差 [   0.332    0.332    0.083    1.327    0.333    1.337    1.331    0.336
    0.017    0.039    0.074    0.213    0.163    0.069   10.31   525.861
  149.833 3136.668]


In [10]:
#计算前18列特征的均值和标准差
mean = np.mean(ds.values[:, :18], axis=0)
std = np.std(ds.values[:, :18], axis=0)


mean1 = np.mean(ds1.values[:, :18], axis=0)
std1 = np.std(ds1.values[:, :18], axis=0)
# 定义上限和下限
upper_limit = mean + 3 * std
lower_limit = mean - 3 * std



upper_limit1 = mean1 + 3 * std
lower_limit1 = mean1 - 3 * std
# 使用布尔索引删除超出上限和下限的行
cleaned_ds = ds[~((ds.values[:, :18] > upper_limit) | (ds.values[:, :18] < lower_limit)).any(axis=1)]


cleaned_ds1 = ds1[~((ds1.values[:, :18] > upper_limit1) | (ds1.values[:, :18] < lower_limit1)).any(axis=1)]
# # 计算特征的均值和标准差
# mean = np.mean(ds.values, axis=0)
# std = np.std(ds.values, axis=0)

# # 定义上限和下限
# upper_limit = mean + 3 * std
# lower_limit = mean - 3 * std

# # 使用布尔索引删除超出上限和下限的行
# cleaned_ds = ds[~((ds.values > upper_limit) | (ds.values < lower_limit)).any(axis=1)]

# 打印清理后数据集的形状
print(cleaned_ds.shape)

print(cleaned_ds1.shape)
cleaned_ds.describe()


(45455, 26)
(2771, 26)


,特征0,特征1,特征2,特征3,特征4,特征5,特征6,特征7,特征8,特征9,...,特征16,特征17,补偿0,补偿1,补偿2,补偿3,补偿4,补偿5,补偿6,补偿7
count,45455.000000,45455.000000,45455.000000,45455.000000,45455.000000,45455.000000,45455.000000,45455.000000,45455.000000,45455.000000,...,45455.000000,45455.000000,45455.000000,45455.000000,45455.000000,45455.000000,45455.000000,45455.000000,45455.000000,45455.000000
mean,1.014642,2.013183,10.510766,7.003799,3.013533,7.014718,2.033679,3.004369,11.744101,12.614738,...,172.279479,1468.611993,0.511533,-0.000280,0.243483,0.014763,-0.344803,0.518227,0.218633,0.525979
std,0.572315,0.573504,0.285374,1.135992,0.563851,1.134305,1.147723,0.572401,0.119637,0.178904,...,10.874075,47.381258,0.387979,0.707994,0.489074,0.317333,0.641076,0.500380,0.713947,1.102260
min,0.000100,1.000000,10.000000,5.000000,2.000100,5.000100,0.000100,2.000100,11.343700,12.015500,...,135.392000,1367.270000,-0.960185,-1.000000,-1.417860,-0.533200,-1.000000,-0.882233,-1.000000,-3.625560
25%,0.524750,1.521700,10.268100,6.037000,2.536300,6.051750,1.045250,2.507600,11.673000,12.507600,...,165.404000,1436.090000,0.266459,-0.707664,-0.097622,-0.143500,-0.872061,0.288231,-0.491356,-0.130062
50%,1.019800,2.021500,10.513000,7.007800,3.021300,7.019900,2.053300,3.006100,11.765700,12.647400,...,173.779000,1453.560000,0.591393,0.002151,0.246505,-0.143500,-0.627452,0.706473,0.478781,0.363849
75%,1.507000,2.506900,10.756150,7.974000,3.494600,7.980150,3.027750,3.496800,11.835500,12.751400,...,180.708500,1489.290000,0.834392,0.706349,0.595703,0.224500,0.182918,0.909821,0.875031,1.122345
max,2.000000,3.000000,11.000000,8.999900,4.000000,9.000000,4.000000,4.000000,11.958200,12.940900,...,193.672000,1642.800000,1.000000,1.000000,1.905170,0.632400,1.000000,1.000000,1.000000,7.422190


In [11]:
# from sklearn.preprocessing import MinMaxScaler

# # 提取特征数据和标签数据
# X = cleaned_ds.values[:, :18]  # 提取前18列作为特征数据
# y = cleaned_ds.values[:, 18:]  # 提取后8列作为标签数据

# # 创建MinMaxScaler对象，并对特征数据进行归一化
# scaler = MinMaxScaler()
# X_normalized = scaler.fit_transform(X)

# # 打印归一化后特征数据的范围（最小值和最大值）
# print('特征数据归一化范围:', np.min(X_normalized), np.max(X_normalized))

# # 打印训练数据集上计算得到的均值和标准差
# print('训练数据集均值:', scaler.mean_)
# print('训练数据集标准差:', scaler.scale_)

# # 将归一化后的特征数据与标签数据重新合并成一个数据集
# normalized_ds = np.concatenate((X_normalized, y), axis=1)
# # 将归一化后的数据集转换为DataFrame
# normalized_ds = pd.DataFrame(normalized_ds)


## 构建训练集和测试集

接着，我们将数据分为训练集和测试集

我们分别获取训练集和测试集的特征以及补偿值：

In [12]:
train_ds=cleaned_ds.sample(frac=0.9,random_state=0)
val_ds=cleaned_ds.drop(train_ds.index)


#训练集
train_features=train_ds.values[:,:18]
train_labels=train_ds.values[:,18:]

#验证集
val_features=val_ds.values[:,:18]
val_labels=val_ds.values[:,18:]

# 测试集
# test_features=test_dataset.values[:,:18]
# test_labels=test_dataset.values[:,18:]

test_features = cleaned_ds1.copy()
test_labels = test_features[['补偿'+str(i) for i in range(8)]].copy()
test_features = test_features.drop(['补偿'+str(i) for i in range(8)], axis=1)

# test_features=cleaned_ds1.values[:,:18]
# test_labels=cleaned_ds1.values[:,18:]

print(train_features.shape,train_labels.shape)
print(val_features.shape,val_labels.shape)
print(test_features.shape,test_labels.shape)

(40910, 18) (40910, 8)
(4545, 18) (4545, 8)
(2771, 18) (2771, 8)


数据的分析和预处理的方法有很多种，我们只展示了一种方法。用户可根据自己的需要使用其他方法。

数据的分析和预处理的方法有很多种，我们只展示了一种方法。用户可根据自己的需要使用其他方法。

## 模型构建

本平台支持基于Tensorflow-Serving的HTTP调用方式：该方式支持任何部署在TensorFlow Serving上的模型

### TensorFlow
首先，我们导入相关的依赖包。

In [13]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from numpy import array
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

print(tf.__version__)

2.8.2


然后，我们开始构建模型

In [15]:
#----------构建模型及训练-----------------
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import optimizers
# model = tf.keras.Sequential([
#     layers.Dense(100, input_dim=train_features.shape[1], activation="relu"),
#     Dropout(0.5),
#     layers.Dense(train_labels.shape[1])
# ])


# 创建Normalizaiton层
normalizer = tf.keras.layers.experimental.preprocessing.Normalization()
# 计算并设置归一化参数
normalizer.adapt(train_features)

# 构建模型
model = tf.keras.Sequential([
    normalizer,  # 归一化层作为第一层
    layers.Dense(300, activation="relu", input_dim=train_features.shape[1]),
    layers.Dense(300, activation="relu"),
    layers.Dense(300, activation="relu"),
    layers.Dense(train_labels.shape[1])
    # normalizer,  # 归一化层作为第一层
    # layers.Dense(100, activation="relu"),
    # layers.Dense(train_labels.shape[1])
])

# optimizers=optimizers.Nadam(learning_rate=0.01)
model.compile(loss="mse", optimizer="adam")  # 根据情况调整参数
model.summary()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normalizat  (None, 18)               37        
 ion)                                                            
                                                                 
 dense_4 (Dense)             (None, 300)               5700      
                                                                 
 dense_5 (Dense)             (None, 300)               90300     
                                                                 
 dense_6 (Dense)             (None, 300)               90300     
                                                                 
 dense_7 (Dense)             (None, 8)                 2408      
                                                                 
Total params: 188,745
Trainable params: 188,708
Non-trainable params: 37
_______________________________________________

## 模型训练
设置模型训练参数进行模型训练

In [16]:
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping

# 设置 EarlyStopping 回调函数，如果验证集的损失不再改善，则停止训练
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.9, patience=5)
model.fit(   # 根据情况调整参数
    train_features,
    train_labels,
    validation_data=(val_features, val_labels),
    epochs=200,
    batch_size=32,
    callbacks=[lr_scheduler]
)


Epoch 1/200
1279/1279 [==============================] - 3s 2ms/step - loss: 0.1354 - val_loss: 0.1205 - lr: 0.0010
Epoch 2/200
1279/1279 [==============================] - 2s 2ms/step - loss: 0.1150 - val_loss: 0.1117 - lr: 0.0010
Epoch 3/200
1279/1279 [==============================] - 2s 2ms/step - loss: 0.1099 - val_loss: 0.1077 - lr: 0.0010
Epoch 4/200
1279/1279 [==============================] - 3s 2ms/step - loss: 0.1067 - val_loss: 0.1069 - lr: 0.0010
Epoch 5/200
1279/1279 [==============================] - 3s 2ms/step - loss: 0.1047 - val_loss: 0.1046 - lr: 0.0010
Epoch 6/200
1279/1279 [==============================] - 3s 2ms/step - loss: 0.1038 - val_loss: 0.1030 - lr: 0.0010
Epoch 7/200
1279/1279 [==============================] - 3s 2ms/step - loss: 0.1030 - val_loss: 0.1030 - lr: 0.0010
Epoch 8/200
1279/1279 [==============================] - 3s 2ms/step - loss: 0.1021 - val_loss: 0.1022 - lr: 0.0010
Epoch 9/200
1279/1279 [==============================] - 3s 2ms/step - l

测试模型训练结果

In [17]:
test_preds = model.predict(test_features)
print("y1 MSE:%.4f" % mean_squared_error(test_labels, test_preds))
print("y1 MSE:%.4f" % len(test_labels), '------',len(test_preds))

y1 MSE:0.0468
y1 MSE:2771.0000 ------ 2771


### 模型部署

误差补偿模型的部署路径为<I>v1/models/slot0/versions/<版本号>/</I> ，且版本号必须为数字。注意，tensorflow-serving在加载模型的时候会自动加载版本号最高的模型，并卸载低版本号的模型。因此，每次部署新部署模型时需要递增版本号。由于我们的系统已经预置了一个低精度版本的模型，并且将版本号设置为1，所以用户在部署自定义模型时应当至少将版本号设置为2。

In [18]:
model_version = 1111111111111
tf.keras.models.save_model(
    model,
    f'/models/slot0/{model_version}/', # v1/models/slot0/为tensorflow-serving的模型根目录
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

2023-08-22 09:42:37.590323: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /models/slot0/1111111111111/assets


注意，tensorflow-serving卸载旧版本模型并加载新版本模型的过程往往需要数十秒的时间，在次期间对模型发送请求会得到“Servable not found for request”的错误。用户可以使用<I>docker logs adjustment-serving-container</I>查看是否已经加载完毕。

接下来我们测试是否部署成功：

In [19]:
import json
import requests
from pprint import pprint

test_features=pd.DataFrame(test_features)
req_data = json.dumps({
            'inputs': test_features.values[:1].tolist()
        })  
print(req_data)
response = requests.post(f'http://fireeye-test-model-container:8501/v1/models/slot0/versions/{model_version}:predict', # 根据部署地址填写
                         data=req_data,
                         headers={"content-type": "application/json"})
if response.status_code != 200:
    raise RuntimeError('Request tf-serving failed: ' + response.text)
resp_data = json.loads(response.text)    
if 'outputs' not in resp_data \
                    or type(resp_data['outputs']) is not list:
    raise ValueError('Malformed tf-serving response')

print(resp_data)
print("{'outputs':",test_labels.values[:1].tolist())

print("y1 MSE:%.4f" % mean_squared_error(test_labels.values[:1].tolist(), resp_data['outputs']))


{"inputs": [[1.4158, 2.9711, 10.7935, 7.5279, 2.3352, 8.1042, 2.3096, 3.3367, 11.8639, 12.7142, 1.8581, 0.3898, 19.8309, 19.771, 0.0001, 1.7768, 171.764, 1434.24]]}
{'outputs': [[0.293064177, -0.683718443, 0.314387083, -0.135694787, -0.841075361, 0.85469383, 0.470254153, 1.90548074]]}
{'outputs': [[0.331511, -0.932553, 0.285048, -0.1435, -0.833982, 0.767568, 0.463969, 1.9048]]
y1 MSE:0.0090


测试成功之后，用户需要在web页面配置相关任务的服务地址，地址的格式为：<I>“http://fireeye-test-model-container:8501/v1/models/slot0/versions/<版本号>:predict ”</I>。